In [1]:
import pandas as pd
import datetime
from os import listdir
import os, os.path
from os.path import isfile, join
import numpy as np
import csv
import re
import pandas as pd
from pm4py.objects.conversion.log import converter as log_converter
from pm4py.objects.log.importer.xes import importer as xes_importer
from pm4py.objects.log.exporter.xes import exporter as xes_exporter
from pm4py.objects.log.util import dataframe_utils
import matplotlib.pyplot as plt
import seaborn as sns

In [5]:
datos_raw = pd.read_excel("./grouped table of annotations.xlsx")
# datos_raw=datos_raw.drop(['Unnamed: 0'], axis='columns')
glosario = pd.read_excel("./OperationCoding (solo cycle time).xlsx")
glosario['General operations'] = glosario['General operations'].fillna(method='pad')
glosario['Specific operations'] =  glosario['Specific operations'].str.split().str.join(' ')
glosario['General operations'] =  glosario['General operations'].str.split().str.join(' ')

# columns=['case:concept:name','General operation','Specific operation','Time','Question','Year', 'Category']
columns=['case:concept:name','concept:name','Specific operation','time:timestamp', 'Report', 'Question','Year', 'Category']
#columna es el case_ID
cont = 0
df_prueba= pd.DataFrame(columns=columns)
for columna in datos_raw.columns:
    acciones_analisis=datos_raw[columna].dropna()
    for accion in acciones_analisis:
        
        accion = re.sub('[\xa0]', ' ', accion)
#         print(' *************************** ', accion)
        string_dividido=accion.split(' ')
        accion = ' '.join(string_dividido)
        for fila in range(len(glosario)):
                       
            if(accion == glosario['Specific operations'][fila]):
                #print('si', accion)
                accionGeneral=glosario['General operations'][fila]
                accionEspecifica=accion
                columnaSeparada=columna.split()
                report=columnaSeparada[0]
                pregunta=columnaSeparada[1]
                año=columnaSeparada[3]
                categoria=datos_raw.loc[0][columna]
                ct=datetime.datetime.now() + datetime.timedelta(days=1)
                new_row={'case:concept:name':columna, 'concept:name':accionGeneral,
                         'Specific operation':accionEspecifica,'time:timestamp':str(ct),
                         'Report':report, 'Question':pregunta, 'Year':año, 'Category':categoria} 
#                 print(cont, new_row['case:concept:name'])
                cont = cont +1
                df_prueba = df_prueba.append(new_row, ignore_index=True)
                
            elif(accion == glosario['General operations'][fila]):
                #print('no', accion)
                accionGeneral=accion
                accionEspecifica=" "
                columnaSeparada=columna.split()
                report=columnaSeparada[0]
                pregunta=columnaSeparada[1]
                año=columnaSeparada[3]
                categoria=datos_raw.loc[0][columna]
                ct=datetime.datetime.now() + datetime.timedelta(days=1)
                new_row={'case:concept:name':columna, 'concept:name':accionGeneral,
                         'Specific operation':accionEspecifica,'time:timestamp':str(ct),
                         'Report':report, 'Question':pregunta,'Year':año, 'Category':categoria} 
#                 print(cont, new_row['case:concept:name'])
                cont = cont +1
                df_prueba = df_prueba.append(new_row, ignore_index=True)
            
                
            
df_prueba

,case:concept:name,concept:name,Specific operation,time:timestamp,Report,Question,Year,Category
0,A1 P2 BPI 2020,Calculate cycle time,Calculate cycle time of the whole process for ...,2021-11-06 08:57:59.867505,A1,P2,2020,Academic
1,A1 P2 BPI 2020,Compare cycle time,,2021-11-06 08:57:59.883136,A1,P2,2020,Academic
2,A2 P2 BPI 2020,Calculate cycle time,Get cycle time from the previous question,2021-11-06 08:57:59.883136,A2,P2,2020,Student
3,A2 P2 BPI 2020,Compare cycle time,,2021-11-06 08:57:59.883136,A2,P2,2020,Student
4,A7 P2 BPI 2020,Calculate throughput,,2021-11-06 08:57:59.898756,A7,P2,2020,Academic
...,...,...,...,...,...,...,...,...
943,A19 P1 BPI 2020,Group traces,Group traces by year,2021-11-06 08:58:03.226313,A19,P1,2020,Professional
944,A19 P1 BPI 2020,Group traces,Group traces by activities,2021-11-06 08:58:03.234406,A19,P1,2020,Professional
945,A19 P1 BPI 2020,Calculate statistics,Calculate Statistics of activities,2021-11-06 08:58:03.236499,A19,P1,2020,Professional
946,A19 P1 BPI 2020,Calculate throughput,,2021-11-06 08:58:03.236499,A19,P1,2020,Professional


In [6]:
reports = pd.read_excel("./Reports BPI Challenge.xlsx")
reports['BPI Challenge'] = reports['BPI Challenge'].fillna(method='pad')
reports

,BPI Challenge,Unnamed: 1,Report,Categoría,Universidad/Organización,Estado,Unnamed: 6,Frecuencia organización,Porcentaje organización
0,2015.0,A1,Ube van der Ham. Benchmarking of Five Dutch Mu...,Profesional,Meijer & Van der Ham Management Consultants,Pendiente (Carlos),En la sección 6.1 se realizan cálculos de cycl...,2.0,3.225806
1,2015.0,A2,"Irene Teinemaa, Anna Leontjeva and Karl-Oskar ...",Estudiante,University of Tartu,Analizado,NaN,1.0,1.612903
2,2015.0,A3,Liese Blevi and Peter Van den Spiegel. Discove...,Profesional,KPMG Advisory,Analizado,NaN,2.0,3.225806
3,2015.0,A4,Scott Buffett and Bruno Emond. Using Sequentia...,Académico,National Research Council Canada,Analizado,NaN,1.0,1.612903
4,2015.0,A5,"Prabhakar M. Dixit, Bart F.A. Hompes, Niek Tax...",Académico,Eindhoven University of Technology,Pendiente (Carlos),En la secciones 5.1 y 5.2 hay detalles de cycl...,5.0,8.064516
...,...,...,...,...,...,...,...,...,...
84,2020.0,A33,"Kushal Poddar, Monika Gupta, Jay Bandlamudi an...",¿Académico?,\nJawaharlal Nehru University\nIBM Research,No,Solo hay:\n-\tSección 3.2 sobre bottlenecks (p...,1.0,1.612903
85,2020.0,A34,"Tatyana Zolotova, Konstantin Mikiev, Daria Zap...",Académico,Eindhoven University of Technology,Analizado,En la sección 6.1 parecen responder a la pregu...,5.0,8.064516
86,2020.0,A35,Analysis of Travel Reimbursement Processes Bas...,Profesional,-,Analizado,NaN,1.0,1.612903
87,2020.0,A36,Chiao-Yun Li and Jingjing Xu Process Conforman...,Académico,"Fraunhofer FIT, Schloss Birlinghoven\nComputer...",No,"En las secciones 6.1, 6.2 y 6.3 dan respuesta...",1.0,1.612903


In [7]:
reports['BPI Challenge'] = reports['BPI Challenge'].astype(int)
reports["BPI Challenge"]=reports["BPI Challenge"].apply(str)
reports

,BPI Challenge,Unnamed: 1,Report,Categoría,Universidad/Organización,Estado,Unnamed: 6,Frecuencia organización,Porcentaje organización
0,2015,A1,Ube van der Ham. Benchmarking of Five Dutch Mu...,Profesional,Meijer & Van der Ham Management Consultants,Pendiente (Carlos),En la sección 6.1 se realizan cálculos de cycl...,2.0,3.225806
1,2015,A2,"Irene Teinemaa, Anna Leontjeva and Karl-Oskar ...",Estudiante,University of Tartu,Analizado,NaN,1.0,1.612903
2,2015,A3,Liese Blevi and Peter Van den Spiegel. Discove...,Profesional,KPMG Advisory,Analizado,NaN,2.0,3.225806
3,2015,A4,Scott Buffett and Bruno Emond. Using Sequentia...,Académico,National Research Council Canada,Analizado,NaN,1.0,1.612903
4,2015,A5,"Prabhakar M. Dixit, Bart F.A. Hompes, Niek Tax...",Académico,Eindhoven University of Technology,Pendiente (Carlos),En la secciones 5.1 y 5.2 hay detalles de cycl...,5.0,8.064516
...,...,...,...,...,...,...,...,...,...
84,2020,A33,"Kushal Poddar, Monika Gupta, Jay Bandlamudi an...",¿Académico?,\nJawaharlal Nehru University\nIBM Research,No,Solo hay:\n-\tSección 3.2 sobre bottlenecks (p...,1.0,1.612903
85,2020,A34,"Tatyana Zolotova, Konstantin Mikiev, Daria Zap...",Académico,Eindhoven University of Technology,Analizado,En la sección 6.1 parecen responder a la pregu...,5.0,8.064516
86,2020,A35,Analysis of Travel Reimbursement Processes Bas...,Profesional,-,Analizado,NaN,1.0,1.612903
87,2020,A36,Chiao-Yun Li and Jingjing Xu Process Conforman...,Académico,"Fraunhofer FIT, Schloss Birlinghoven\nComputer...",No,"En las secciones 6.1, 6.2 y 6.3 dan respuesta...",1.0,1.612903


In [8]:
reports["Unnamed: 1"]=reports["Unnamed: 1"].apply(str)
reports['Tupla'] = reports['Unnamed: 1'] + reports["BPI Challenge"]

In [9]:
df_prueba['Tupla'] = df_prueba['Report'] + df_prueba["Year"]

In [10]:
sLength = len(df_prueba['Tupla'])
df_prueba['Organization'] = pd.Series(np.random.randn(sLength), index=df_prueba.index)
df_prueba

,case:concept:name,concept:name,Specific operation,time:timestamp,Report,Question,Year,Category,Tupla,Organization
0,A1 P2 BPI 2020,Calculate cycle time,Calculate cycle time of the whole process for ...,2021-11-06 08:57:59.867505,A1,P2,2020,Academic,A12020,-1.414627
1,A1 P2 BPI 2020,Compare cycle time,,2021-11-06 08:57:59.883136,A1,P2,2020,Academic,A12020,0.672800
2,A2 P2 BPI 2020,Calculate cycle time,Get cycle time from the previous question,2021-11-06 08:57:59.883136,A2,P2,2020,Student,A22020,0.394505
3,A2 P2 BPI 2020,Compare cycle time,,2021-11-06 08:57:59.883136,A2,P2,2020,Student,A22020,1.431669
4,A7 P2 BPI 2020,Calculate throughput,,2021-11-06 08:57:59.898756,A7,P2,2020,Academic,A72020,-0.446423
...,...,...,...,...,...,...,...,...,...,...
943,A19 P1 BPI 2020,Group traces,Group traces by year,2021-11-06 08:58:03.226313,A19,P1,2020,Professional,A192020,-0.297816
944,A19 P1 BPI 2020,Group traces,Group traces by activities,2021-11-06 08:58:03.234406,A19,P1,2020,Professional,A192020,-1.417589
945,A19 P1 BPI 2020,Calculate statistics,Calculate Statistics of activities,2021-11-06 08:58:03.236499,A19,P1,2020,Professional,A192020,-0.833178
946,A19 P1 BPI 2020,Calculate throughput,,2021-11-06 08:58:03.236499,A19,P1,2020,Professional,A192020,0.525304


In [11]:
df_prueba['prueba']= ""
df_prueba.drop(['Organization', 'prueba'], axis=1)

,case:concept:name,concept:name,Specific operation,time:timestamp,Report,Question,Year,Category,Tupla
0,A1 P2 BPI 2020,Calculate cycle time,Calculate cycle time of the whole process for ...,2021-11-06 08:57:59.867505,A1,P2,2020,Academic,A12020
1,A1 P2 BPI 2020,Compare cycle time,,2021-11-06 08:57:59.883136,A1,P2,2020,Academic,A12020
2,A2 P2 BPI 2020,Calculate cycle time,Get cycle time from the previous question,2021-11-06 08:57:59.883136,A2,P2,2020,Student,A22020
3,A2 P2 BPI 2020,Compare cycle time,,2021-11-06 08:57:59.883136,A2,P2,2020,Student,A22020
4,A7 P2 BPI 2020,Calculate throughput,,2021-11-06 08:57:59.898756,A7,P2,2020,Academic,A72020
...,...,...,...,...,...,...,...,...,...
943,A19 P1 BPI 2020,Group traces,Group traces by year,2021-11-06 08:58:03.226313,A19,P1,2020,Professional,A192020
944,A19 P1 BPI 2020,Group traces,Group traces by activities,2021-11-06 08:58:03.234406,A19,P1,2020,Professional,A192020
945,A19 P1 BPI 2020,Calculate statistics,Calculate Statistics of activities,2021-11-06 08:58:03.236499,A19,P1,2020,Professional,A192020
946,A19 P1 BPI 2020,Calculate throughput,,2021-11-06 08:58:03.236499,A19,P1,2020,Professional,A192020


In [12]:
# lista=[]
df_prueba['Organization'] = ""
for d in range(len(df_prueba)):
    for r in range(len(reports)):
        if(df_prueba.loc[d]['Tupla'] == reports.loc[r]['Tupla']):
#             print('si')
#             print(reports.iloc[r]['Universidad/Organización'])
#             lista.append(reports.loc[r]['Universidad/Organización'])
             df_prueba.iloc[d]['Organization'] = reports.loc[r]['Universidad/Organización']

In [13]:
df_prueba.drop(['prueba'], axis=1)

,case:concept:name,concept:name,Specific operation,time:timestamp,Report,Question,Year,Category,Tupla,Organization
0,A1 P2 BPI 2020,Calculate cycle time,Calculate cycle time of the whole process for ...,2021-11-06 08:57:59.867505,A1,P2,2020,Academic,A12020,"Technische Universität, Berlin"
1,A1 P2 BPI 2020,Compare cycle time,,2021-11-06 08:57:59.883136,A1,P2,2020,Academic,A12020,"Technische Universität, Berlin"
2,A2 P2 BPI 2020,Calculate cycle time,Get cycle time from the previous question,2021-11-06 08:57:59.883136,A2,P2,2020,Student,A22020,"Faculty of Science, Utrecht University"
3,A2 P2 BPI 2020,Compare cycle time,,2021-11-06 08:57:59.883136,A2,P2,2020,Student,A22020,"Faculty of Science, Utrecht University"
4,A7 P2 BPI 2020,Calculate throughput,,2021-11-06 08:57:59.898756,A7,P2,2020,Academic,A72020,German Research Center for Artificial Intellig...
...,...,...,...,...,...,...,...,...,...,...
943,A19 P1 BPI 2020,Group traces,Group traces by year,2021-11-06 08:58:03.226313,A19,P1,2020,Professional,A192020,Sberbank
944,A19 P1 BPI 2020,Group traces,Group traces by activities,2021-11-06 08:58:03.234406,A19,P1,2020,Professional,A192020,Sberbank
945,A19 P1 BPI 2020,Calculate statistics,Calculate Statistics of activities,2021-11-06 08:58:03.236499,A19,P1,2020,Professional,A192020,Sberbank
946,A19 P1 BPI 2020,Calculate throughput,,2021-11-06 08:58:03.236499,A19,P1,2020,Professional,A192020,Sberbank


In [14]:
df_report_year = df_prueba.groupby(by=['Question', 'Year'])
for name, group in df_report_year:
    operations = group['concept:name'].to_list()
    

In [15]:
df_prueba['concept:name'].to_list()

['Calculate cycle time',
 'Compare cycle time',
 'Calculate cycle time',
 'Compare cycle time',
 'Calculate throughput',
 'Comparative of throughput',
 'Comparative of throughput',
 'Represent scatterplot',
 'Represent boxplots of cycle time',
 'Compare cycle time',
 'Calculate throughput',
 'Preprocess the traces of the logs',
 'Calculate throughput',
 'Calculate cycle time',
 'Compare cycle time',
 'Identify specific subprocesses',
 'Calculate statistics',
 'Calculate cycle time',
 'Compare cycle time',
 'Calculate cycle time',
 'Calculate cycle time',
 'Compare cycle time',
 'Calculate cycle time',
 'Represent histograms',
 'Compare cycle time',
 'Calculate cycle time',
 'Calculate number of elements',
 'Calculate number of elements',
 'Calculate average of activities per trace',
 'Calculate percentages',
 'Calculate statistics',
 'Group traces',
 'Represent histograms',
 'Represent bar charts',
 'Represent histograms',
 'Filter traces',
 'Compare cycle time',
 'Calculate cycle time

In [16]:
df_jaccard['case:concept:name'] = 'hola'
df_jaccard

NameError: name 'df_jaccard' is not defined

In [ ]:
df_analisis = df_prueba.groupby(by=['case:concept:name'])
columns=['case:concept:name','Operations','Organization', 'Year', 'Question']
df_agrupado = pd.DataFrame(columns=columns)
for name, group in df_analisis:
    org = group.iloc[0]['Organization']
    year = group.iloc[0]['Year']
    question = group.iloc[0]['Question']
    lista = group['concept:name'].to_list()
        
    new_row={'case:concept:name':name, 'Operations':lista, 'Organization':org, 'Year':year, 'Question':question} 
    df_agrupado = df_agrupado.append(new_row, ignore_index=True)

In [3]:
df_agrupado

NameError: name 'df_agrupado' is not defined

In [4]:
group

NameError: name 'group' is not defined

In [107]:
def jaccardIndex(list1,list2):
    set1=set(list1)
    set2=set(list2)
    intersection = set1.intersection(set2)
    tamIntersection=len(list(intersection))
    union = set1.union(set2)
    tamUnion=len(list(union))
#     print("similitud: "+str(float(tamIntersection)/tamUnion))
    return float(tamIntersection)/tamUnion

In [125]:
df2 = df_agrupado.groupby(by=['Year', 'Question'])
columns=['analisis','organizaciones','jaccard']
df_jaccard = pd.DataFrame(columns=columns)

for name, group in df2:
    for i in range(len(group)-1):
        if(group.iloc[i]['Organization'] != 'Sberbank'):
            org1 = 'No_Sberbank'
        else:
            org1 = 'Sberbank'
        
        for j in range(i+1,len(group)-1):
            grupo = group.iloc[i]['Operations']
            analisis = group.iloc[i]['case:concept:name'] + "-" + group.iloc[j]['case:concept:name']
                
            if(group.iloc[j]['Organization'] != 'Sberbank'):
                org2 = 'No_Sberbank'
            else:
                org2 = 'Sberbank'
            org = org1 + "-" + org2
            jaccard = jaccardIndex(grupo,group.iloc[j]['Operations'])
            new_row={'analisis':analisis,'organizaciones':org,'jaccard': jaccard} 
            df_jaccard = df_jaccard.append(new_row, ignore_index=True)
        


In [129]:
df_jaccard = df_jaccard.replace({'Sberbank-No_Sberbank':'No_Sberbank-Sberbank'})

In [130]:
df_jaccard.groupby(by=['organizaciones']).mean()

,jaccard
organizaciones,
No_Sberbank-No_Sberbank,0.285992
No_Sberbank-Sberbank,0.260242
Sberbank-Sberbank,0.205340


In [134]:
from scipy import stats

group1 =  df_jaccard['organizaciones'] == 'Sberbank-Sberbank'
group2 =  df_jaccard['organizaciones'] == 'No_Sberbank-Sberbank'
group3 =  df_jaccard['organizaciones'] == 'No_Sberbank-No_Sberbank'
stats.kruskal(group1, group2, group3)

# (statistic=6.2878, pvalue=0.0431)

KruskalResult(statistic=180.16235346896303, pvalue=7.55513137923488e-40)

In [135]:
stats.kruskal(group1, group2)

KruskalResult(statistic=78.6449978930064, pvalue=7.433672392474974e-19)

In [136]:
stats.kruskal(group2, group3)

KruskalResult(statistic=22.778515966754437, pvalue=1.817865766018157e-06)

In [137]:
stats.kruskal(group1, group3)

KruskalResult(statistic=179.9747145970096, pvalue=4.908413575113177e-41)

In [142]:
def overlapIndex(list1,list2):
    set1=set(list1)
    set2=set(list2)
    intersection = set1.intersection(set2)
    len1=len(list(set1))
    len2=len(list(set2))
#     print(float(len(list(intersection))))
#     print(min([len1,len2]))
    overlapIndex=float(len(list(intersection)))/min([len1,len2])
    return overlapIndex

In [147]:
df2 = df_agrupado.groupby(by=['Year', 'Question'])
columns=['analisis','organizaciones','overlap']
df_overlap = pd.DataFrame(columns=columns)

for name, group in df2:
    for i in range(len(group)-1):
        if(group.iloc[i]['Organization'] != 'Sberbank'):
            org1 = 'No_Sberbank'
        else:
            org1 = 'Sberbank'
        
        for j in range(i+1,len(group)-1):
            grupo = group.iloc[i]['Operations']
            analisis = group.iloc[i]['case:concept:name'] + "-" + group.iloc[j]['case:concept:name']
                
            if(group.iloc[j]['Organization'] != 'Sberbank'):
                org2 = 'No_Sberbank'
            else:
                org2 = 'Sberbank'
            org = org1 + "-" + org2
            overlap = overlapIndex(grupo,group.iloc[j]['Operations'])
            new_row={'analisis':analisis,'organizaciones':org,'overlap': overlap} 
            df_overlap = df_overlap.append(new_row, ignore_index=True)
        


In [148]:
df_overlap.groupby(by=['organizaciones']).mean()

,overlap
organizaciones,
No_Sberbank-No_Sberbank,0.567003
No_Sberbank-Sberbank,0.500204
Sberbank-No_Sberbank,0.522383
Sberbank-Sberbank,0.421026
